In [1]:
import json
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from alpha_vantage.timeseries import TimeSeries

In [2]:
class ScriptData():
    """ 
    Class implements methods to load data, process data 
    and convert into DataFrame.
    """
    def __init__(self):
        """ 
        Constructor to load api key and 
        create a object to load script data.
        """

        self.data = dict()
        self.api_key = "234IJADSE4UWFFQW"
        self.columns = ["timestamp", "open", "high", "low", "close", "volume"]
        self.ts = TimeSeries(key=self.api_key)
        
    def fetch_intraday_data(self, script):
        """ 
        Load intraday data from script 
        Args: 
            script: script name(str)

        Returns: data dictionary
        """

        self.data[script], _ = self.ts.get_intraday(script)
        return self.data[script]

    def convert_intraday_data(self, script):
        """
        Convert data dictionary into DataFrame.
        Args: 
            script: script name(str) 

        Returns: DataFrame
        """

        if script not in self.data.keys():
            self.fetch_intraday_data(script)

        if type(self.data[script]) != dict:
            return self.data[script] 
        
        ts_format = "%Y-%m-%d %H:%M:%S"
        data_entries = [[datetime.strptime(key, ts_format)] + list(value.values()) 
                    for key, value in self.data[script].items()]
        
        self.data[script] = pd.DataFrame(data_entries, columns=self.columns)
        self.data[script] = self.data[script].sort_values('timestamp', ignore_index=True)
        self.data[script]['close'] = self.data[script]['close'].apply(lambda x : float(x))
        return self.data[script]

    def __getitem__(self, script):
        """
        Get the DataFrame for a script.
        Args: 
            script: script name(str) 
        
        Returns: DataFrame
        """

        return self.data[script]
    
    def __setitem__(self, script, data):
        """
        Set data for a script.
        Args: 
            data = dict or DataFrame
            script: script name(str) 
        
        Returns: None
        """

        self.data[script] = data
    
    def __contains__(self, script):
        """
        Check if data exits for a script.
        Args: 
            script: script name(str) 
        
        Returns: DataFrame
        """

        return script in self.data.keys()

In [3]:
def indicator1(df, timeperoid=5):
        """ 
        Calcualte SMA for a given timeperiod on close data.
        Args: 
            df : DataFrame
            timeperiod: window size to calulcate SMA (int)
        
        Returns: DataFrame
        """

        df['indicator'] = df['close'].rolling(timeperoid, min_periods=timeperoid).mean()
        df = df[['timestamp', 'indicator']]
        return df

In [4]:
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-16 11:15:00,97.5500,97.6800,97.0300,97.19,1609968
1,2023-02-16 11:30:00,97.1900,97.2000,96.6900,96.81,1498373
2,2023-02-16 11:45:00,96.8200,96.9700,96.5300,96.83,1200905
3,2023-02-16 12:00:00,96.8300,97.0900,96.8150,96.95,714856
4,2023-02-16 12:15:00,96.9450,96.9900,96.5300,96.88,900381
...,...,...,...,...,...,...
95,2023-02-17 19:00:00,94.2500,94.2900,94.2500,94.29,1887
96,2023-02-17 19:15:00,94.2900,94.3200,94.2700,94.27,3252
97,2023-02-17 19:30:00,94.2600,94.3100,94.2300,94.26,5439
98,2023-02-17 19:45:00,94.3200,94.3200,94.2300,94.26,1592


In [5]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-16 11:15:00,155.1600,155.4600,154.8889,155.0800,2176839
1,2023-02-16 11:30:00,155.0803,155.3999,154.9200,155.2499,1484408
2,2023-02-16 11:45:00,155.2500,155.4450,155.0501,155.2850,1220633
3,2023-02-16 12:00:00,155.2800,155.5000,155.0100,155.0700,1559793
4,2023-02-16 12:15:00,155.0650,155.5000,155.0200,155.4301,1148890
...,...,...,...,...,...,...
95,2023-02-17 19:00:00,152.6200,152.6300,152.5600,152.6200,5858
96,2023-02-17 19:15:00,152.6200,152.6300,152.5500,152.5500,4504
97,2023-02-17 19:30:00,152.5500,152.5700,152.5500,152.5700,2655
98,2023-02-17 19:45:00,152.5600,152.6000,152.5500,152.5600,5131


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
indicator1(script_data['GOOGL'], timeperoid=5)

,timestamp,indicator
0,2023-02-16 11:15:00,NaN
1,2023-02-16 11:30:00,NaN
2,2023-02-16 11:45:00,NaN
3,2023-02-16 12:00:00,NaN
4,2023-02-16 12:15:00,96.932
...,...,...
95,2023-02-17 19:00:00,94.202
96,2023-02-17 19:15:00,94.216
97,2023-02-17 19:30:00,94.240
98,2023-02-17 19:45:00,94.266


In [10]:
indicator1(script_data['AAPL'], timeperoid=5)

,timestamp,indicator
0,2023-02-16 11:15:00,NaN
1,2023-02-16 11:30:00,NaN
2,2023-02-16 11:45:00,NaN
3,2023-02-16 12:00:00,NaN
4,2023-02-16 12:15:00,155.223
...,...,...
95,2023-02-17 19:00:00,152.574
96,2023-02-17 19:15:00,152.584
97,2023-02-17 19:30:00,152.598
98,2023-02-17 19:45:00,152.582


In [11]:
class Strategy(ScriptData):
    """ 
    class implments methods to get script data and 
    get signal data i.e.'BUY' and 'SELL' signal from scipt data.
    """

    def __init__(self, script):
        super().__init__()
        self.script = script

    def get_script_data(self):
        """
        Fetche the intrady data for a script

        Returns: None
        """

        self.fetch_intraday_data(self.script)
        self.convert_intraday_data(self.script)

    def get_signals(self):
        """
        Prcoess data to find 'BUY' and 'SELL' signal.
        
        Returns: Dataframe with columns 'timestamp' and 'singnal'.
        """

        df = self.data[self.script].loc[:,['timestamp', 'close']]
        df['indicator_data'] = indicator1(self.data[self.script])['indicator']
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        close_data = df['close'].values
        indicator_data = df['indicator_data'].values

        cut_data =  np.diff(np.sign(indicator_data - close_data))
        cut_indices = np.argwhere(cut_data).flatten()
        
        df = df.loc[cut_indices,:]
        signal = np.where(cut_data[np.where(cut_data != 0)]<0, 'BUY', 'SELL')
        df['signal'] = signal
        df.drop(['close', 'indicator_data'], axis=1, inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df

In [12]:
strategy = Strategy('NVDA')

In [13]:
strategy.get_script_data()

In [14]:
strategy.get_signals()

,timestamp,signal
0,2023-02-16 13:00:00,SELL
1,2023-02-16 13:30:00,BUY
2,2023-02-16 14:15:00,SELL
3,2023-02-16 19:30:00,BUY
4,2023-02-16 20:00:00,SELL
5,2023-02-17 04:15:00,BUY
6,2023-02-17 04:30:00,SELL
7,2023-02-17 06:15:00,BUY
8,2023-02-17 07:15:00,SELL
9,2023-02-17 07:30:00,BUY


In [15]:
# plot camdle stick charts using plotly.py
import plotly.graph_objects as go

# store the GOOGL script data in df
df = script_data['GOOGL']


fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig.show()